In [1]:
!nvidia-smi

Tue Jan  2 23:10:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [28]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.1.2 --progress-bar off
!pip install -qqq transformers==4.36.2 --progress-bar off
!pip install -qqq accelerate==0.25.0 --progress-bar off
!pip install -qqq bitsandbytes==0.41.3.post2 --progress-bar off
!pip install -qqq tokenizers==0.15.0 --progress-bar off
!pip install -qqq sentencepiece==0.1.99 --progress-bar off
!pip install -qqq sentence-transformers==2.2.2 --progress-bar off
!pip install -qqq llama-index==0.9.24 --progress-bar off

NotImplementedError: ignored

## RAG From Scratch

In [3]:
exercises = [
    {
        "name": "deadlift",
        "muscle_groups": ["back", "hamstrings"],
        "description": "Stand with feet hip-width apart, gripping the barbell with hands shoulder-width apart. Lift the barbell by straightening your legs and back.",
    },
    {
        "name": "bench press",
        "muscle_groups": ["chest", "triceps", "shoulders"],
        "description": "Lie on a bench, feet flat on the ground. Push the barbell from chest level upwards until arms are fully extended.",
    },
    {
        "name": "squat",
        "muscle_groups": ["quadriceps", "glutes", "lower back"],
        "description": "Stand with feet shoulder-width apart. Bend knees and hips to lower down as if sitting back into a chair, then return to standing.",
    },
    {
        "name": "pull-up",
        "muscle_groups": ["back", "biceps"],
        "description": "Hang from a bar with hands slightly wider than shoulder-width. Pull body up until chin is above the bar, then lower back down.",
    },
    {
        "name": "shoulder press",
        "muscle_groups": ["shoulders", "triceps"],
        "description": "Sit or stand with a dumbbell in each hand at shoulder height. Press the weights upwards until arms are extended overhead.",
    },
    {
        "name": "barbell row",
        "muscle_groups": ["back", "biceps"],
        "description": "Bend forward at the waist, holding a barbell with hands shoulder-width apart. Pull the bar towards your lower chest.",
    },
    {
        "name": "lunges",
        "muscle_groups": ["quadriceps", "glutes", "hamstrings"],
        "description": "Step forward with one leg and lower your body until both knees are bent at 90-degree angles. Push back up to the starting position.",
    },
    {
        "name": "leg press",
        "muscle_groups": ["quadriceps", "glutes", "hamstrings"],
        "description": "Sit in a leg press machine with feet hip-width apart on the platform. Extend legs fully, then return to starting position.",
    },
    {
        "name": "bicep curl",
        "muscle_groups": ["biceps"],
        "description": "Stand holding a dumbbell in each hand with arms extended. Curl the weights towards your shoulders, then lower them back down.",
    },
    {
        "name": "tricep dip",
        "muscle_groups": ["triceps"],
        "description": "Using parallel bars or a bench, lower your body by bending your arms, then push back up to the starting position.",
    },
    {
        "name": "leg curl",
        "muscle_groups": ["hamstrings"],
        "description": "Lie face down on a leg curl machine and curl the weight towards your buttocks, then lower it back down.",
    },
    {
        "name": "calf raise",
        "muscle_groups": ["calves"],
        "description": "Stand on the edge of a step or platform. Rise onto your toes, then lower back down below the level of the step.",
    },
    {
        "name": "lat pulldown",
        "muscle_groups": ["back", "biceps"],
        "description": "Sit at a lat pulldown machine, grab the bar wider than shoulder-width, and pull the bar down to your upper chest.",
    },
    {
        "name": "pec deck",
        "muscle_groups": ["chest"],
        "description": "Sit in a pec deck machine with arms on the pads. Bring your arms together in front of you, then return to the starting position.",
    },
    {
        "name": "incline bench press",
        "muscle_groups": ["upper chest", "shoulders", "triceps"],
        "description": "Lie on an incline bench. Push the barbell from chest level upwards until arms are fully extended.",
    },
    {
        "name": "front squat",
        "muscle_groups": ["quadriceps", "glutes", "upper back"],
        "description": "Stand with feet shoulder-width apart, holding a barbell in front of your shoulders. Lower into a squat, then return to standing.",
    },
    {
        "name": "Romanian deadlift",
        "muscle_groups": ["hamstrings", "lower back", "glutes"],
        "description": "Hold a barbell in front of your thighs. Bend at hips to lower the bar, keeping your back straight, then return to standing.",
    },
    {
        "name": "t-bar row",
        "muscle_groups": ["back", "biceps"],
        "description": "Straddle a T-bar row machine and lift the bar by pulling it towards your chest while keeping your back straight.",
    },
    {
        "name": "overhead tricep extension",
        "muscle_groups": ["triceps"],
        "description": "Hold a dumbbell with both hands above your head. Bend your elbows to lower the weight behind your head, then extend your arms back up.",
    },
    {
        "name": "seated leg extension",
        "muscle_groups": ["quadriceps"],
        "description": "Sit on a leg extension machine with your legs under the pad. Extend your legs out in front of you, then return to the starting position.",
    },
    {
        "name": "lying tricep extension",
        "muscle_groups": ["triceps"],
        "description": "Lie on a bench holding a barbell. Lower the bar to your forehead, then extend your arms to raise it back up.",
    },
    {
        "name": "seated cable row",
        "muscle_groups": ["back", "biceps"],
        "description": "Sit at a cable row machine, grab the handle, and pull it towards your waist while keeping your back straight.",
    },
    {
        "name": "hammer curl",
        "muscle_groups": ["biceps"],
        "description": "Stand holding dumbbells with palms facing your body. Curl the weights up, keeping your palms facing each other.",
    },
    {
        "name": "reverse fly",
        "muscle_groups": ["shoulders", "upper back"],
        "description": "Bend forward at the waist, holding dumbbells. Raise your arms to the side until they are parallel with the floor.",
    },
    {
        "name": "face pull",
        "muscle_groups": ["shoulders", "upper back"],
        "description": "Stand facing a cable machine with a rope attachment. Pull the rope towards your face while separating your hands.",
    },
    {
        "name": "sumo deadlift",
        "muscle_groups": ["glutes", "inner thighs", "lower back"],
        "description": "Stand with feet wider than shoulder-width, holding a barbell. Keep your back straight and lift the barbell by straightening your legs.",
    },
    {
        "name": "hack squat",
        "muscle_groups": ["quadriceps", "glutes", "calves"],
        "description": "Stand in a hack squat machine with your back against the pad. Lower down into a squat, then push back up to standing.",
    },
    {
        "name": "hip thrust",
        "muscle_groups": ["glutes", "hamstrings"],
        "description": "Sit on the ground with a bench behind you. Place a barbell over your hips and thrust upward, pushing your hips towards the sky.",
    },
    {
        "name": "farmers walk",
        "muscle_groups": ["forearms", "shoulders", "traps"],
        "description": "Hold a heavy weight in each hand. Walk a set distance or for a set time while keeping your posture upright.",
    },
    {
        "name": "decline bench press",
        "muscle_groups": ["lower chest", "triceps"],
        "description": "Lie on a decline bench. Push the barbell from chest level upwards until arms are fully extended.",
    },
]

In [4]:
def text_to_words(text: str) -> set[str]:
    return set(text.lower().split())


def jaccard_similarity(query, text) -> float:
    query_words = text_to_words(query)
    text_words = text_to_words(text)

    intersection = query_words.intersection(text_words)
    union = query_words.union(text_words)

    return len(intersection) / len(union)

In [5]:
def retrieve(query, documents, k: int = 3):
    scores = [(jaccard_similarity(query, doc), doc) for doc in documents]
    scores.sort(reverse=True, key=lambda x: x[0])
    return scores[:k]

In [6]:
corpus = [
    f'{e["name"]} ( muscle groups: {", ".join(e["muscle_groups"])} ). {e["description"]}'
    for e in exercises
]
corpus[:2]

['deadlift ( muscle groups: back, hamstrings ). Stand with feet hip-width apart, gripping the barbell with hands shoulder-width apart. Lift the barbell by straightening your legs and back.',
 'bench press ( muscle groups: chest, triceps, shoulders ). Lie on a bench, feet flat on the ground. Push the barbell from chest level upwards until arms are fully extended.']

In [7]:
retrieve("barbell back glutes", corpus, k=3)

[(0.1,
  'Romanian deadlift ( muscle groups: hamstrings, lower back, glutes ). Hold a barbell in front of your thighs. Bend at hips to lower the bar, keeping your back straight, then return to standing.'),
 (0.06666666666666667,
  'front squat ( muscle groups: quadriceps, glutes, upper back ). Stand with feet shoulder-width apart, holding a barbell in front of your shoulders. Lower into a squat, then return to standing.'),
 (0.06451612903225806,
  'sumo deadlift ( muscle groups: glutes, inner thighs, lower back ). Stand with feet wider than shoulder-width, holding a barbell. Keep your back straight and lift the barbell by straightening your legs.')]

In [8]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
    TextStreamer,
    pipeline,
)

MODEL_NAME = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, device_map="auto", torch_dtype=torch.float16
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 256
generation_config.repetition_penalty = 1.18
generation_config.temperature = 0.0000001
generation_config.do_sample = True

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

pytorch_model-00006-of-00008.bin:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

pytorch_model-00007-of-00008.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

pytorch_model-00008-of-00008.bin:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [9]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    streamer=streamer,
)

In [10]:
prompt_template = """
Write a workout plan for training legs - glutes, quadriceps and hamstrings
Use the format:

exercise - sets x reps

Choose from the exercises:
```
{exercises_text}
```

Make sure that the number of exercises, sets, and reps follow best practices
for a beginner
"""

found_exercises = retrieve(prompt_template, corpus, k=10)

In [11]:
found_exercises

[(0.08771929824561403,
  'hip thrust ( muscle groups: glutes, hamstrings ). Sit on the ground with a bench behind you. Place a barbell over your hips and thrust upward, pushing your hips towards the sky.'),
 (0.08620689655172414,
  'leg press ( muscle groups: quadriceps, glutes, hamstrings ). Sit in a leg press machine with feet hip-width apart on the platform. Extend legs fully, then return to starting position.'),
 (0.0847457627118644,
  'seated leg extension ( muscle groups: quadriceps ). Sit on a leg extension machine with your legs under the pad. Extend your legs out in front of you, then return to the starting position.'),
 (0.07272727272727272,
  'leg curl ( muscle groups: hamstrings ). Lie face down on a leg curl machine and curl the weight towards your buttocks, then lower it back down.'),
 (0.07142857142857142,
  'deadlift ( muscle groups: back, hamstrings ). Stand with feet hip-width apart, gripping the barbell with hands shoulder-width apart. Lift the barbell by straighteni

In [12]:
exercises_text = "\n\n".join([e for _, e in found_exercises])
print(exercises_text)

hip thrust ( muscle groups: glutes, hamstrings ). Sit on the ground with a bench behind you. Place a barbell over your hips and thrust upward, pushing your hips towards the sky.

leg press ( muscle groups: quadriceps, glutes, hamstrings ). Sit in a leg press machine with feet hip-width apart on the platform. Extend legs fully, then return to starting position.

seated leg extension ( muscle groups: quadriceps ). Sit on a leg extension machine with your legs under the pad. Extend your legs out in front of you, then return to the starting position.

leg curl ( muscle groups: hamstrings ). Lie face down on a leg curl machine and curl the weight towards your buttocks, then lower it back down.

deadlift ( muscle groups: back, hamstrings ). Stand with feet hip-width apart, gripping the barbell with hands shoulder-width apart. Lift the barbell by straightening your legs and back.

sumo deadlift ( muscle groups: glutes, inner thighs, lower back ). Stand with feet wider than shoulder-width, hol

In [13]:
prompt_text = prompt_template.format(exercises_text=exercises_text)
print(prompt_text)


Write a workout plan for training legs - glutes, quadriceps and hamstrings
Use the format:

exercise - sets x reps

Choose from the exercises:
```
hip thrust ( muscle groups: glutes, hamstrings ). Sit on the ground with a bench behind you. Place a barbell over your hips and thrust upward, pushing your hips towards the sky.

leg press ( muscle groups: quadriceps, glutes, hamstrings ). Sit in a leg press machine with feet hip-width apart on the platform. Extend legs fully, then return to starting position.

seated leg extension ( muscle groups: quadriceps ). Sit on a leg extension machine with your legs under the pad. Extend your legs out in front of you, then return to the starting position.

leg curl ( muscle groups: hamstrings ). Lie face down on a leg curl machine and curl the weight towards your buttocks, then lower it back down.

deadlift ( muscle groups: back, hamstrings ). Stand with feet hip-width apart, gripping the barbell with hands shoulder-width apart. Lift the barbell by 

In [14]:
SYSTEM_PROMPT = """
You're an expert BodyBuilding Coach with 20+ years of experience with
training all types of atheletes. You always follow foundational principles when
designing workout plans. Each workout must have 4 exercises at maximum.
"""

prompt_text = SYSTEM_PROMPT + prompt_text

messages = [
    {"role": "user", "content": prompt_text},
]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
print(prompt)

<s>[INST] 
You're an expert BodyBuilding Coach with 20+ years of experience with
training all types of atheletes. You always follow foundational principles when
designing workout plans. Each workout must have 4 exercises at maximum.

Write a workout plan for training legs - glutes, quadriceps and hamstrings
Use the format:

exercise - sets x reps

Choose from the exercises:
```
hip thrust ( muscle groups: glutes, hamstrings ). Sit on the ground with a bench behind you. Place a barbell over your hips and thrust upward, pushing your hips towards the sky.

leg press ( muscle groups: quadriceps, glutes, hamstrings ). Sit in a leg press machine with feet hip-width apart on the platform. Extend legs fully, then return to starting position.

seated leg extension ( muscle groups: quadriceps ). Sit on a leg extension machine with your legs under the pad. Extend your legs out in front of you, then return to the starting position.

leg curl ( muscle groups: hamstrings ). Lie face down on a leg cu

In [15]:
output = llm(prompt)

Here is a sample workout plan for training legs - glutes, quadriceps, and hamstrings:

1. Hip Thrust - 3 sets of 8-12 reps
2. Leg Press - 3 sets of 8-12 reps
3. Seated Leg Extension - 3 sets of 8-12 reps
4. Lat Pulldown - 3 sets of 8-12 reps

This workout plan includes four exercises that target different parts of the legs and glutes. The first three exercises focus on building strength and endurance in the quadriceps, hamstrings, and glutes, respectively. The fourth exercise, the lat pulldown, targets the back muscles which help support the legs during movements like squats and lunges. It's important to note that this is just a sample workout plan and should be adjusted based on individual fitness levels and goals.


## Embeddings

In [16]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("BAAI/bge-base-en-v1.5")

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.2k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [17]:
prompt_template = """
Write a workout plan for training legs - glutes, quadriceps and hamstrings
Use the format:

exercise - sets x reps

Choose from the exercises:
```
{exercises_text}
```

Make sure that the number of exercises, sets, and reps follow best practices
for a beginner
"""

In [18]:
prompts = [prompt_template]
prompt_embeddings = embedding_model.encode(prompts, normalize_embeddings=True)
prompt_embeddings.shape

(1, 768)

In [19]:
list(embedding_model.children())

[Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel ,
 Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False}),
 Normalize()]

In [20]:
corpus_embeddings = embedding_model.encode(corpus, normalize_embeddings=True)
similarities = prompt_embeddings @ corpus_embeddings.T

In [21]:
similarities

array([[0.64083433, 0.5763533 , 0.6854795 , 0.55268663, 0.57409495,
        0.5585253 , 0.7137631 , 0.7238476 , 0.55645543, 0.58007133,
        0.6265472 , 0.613955  , 0.5163127 , 0.5264634 , 0.54215574,
        0.6710038 , 0.63459265, 0.5417024 , 0.5506798 , 0.63438725,
        0.5459614 , 0.52956176, 0.56054866, 0.57462955, 0.4805007 ,
        0.606614  , 0.6444622 , 0.6720909 , 0.5213034 , 0.5432125 ]],
      dtype=float32)

In [22]:
scores = similarities.flatten().tolist()

In [23]:
scores = [(score, document) for score, document in zip(scores, corpus)]
scores = sorted(scores, key=lambda x: x[0], reverse=True)
scores[:5]

[(0.7238476276397705,
  'leg press ( muscle groups: quadriceps, glutes, hamstrings ). Sit in a leg press machine with feet hip-width apart on the platform. Extend legs fully, then return to starting position.'),
 (0.7137631177902222,
  'lunges ( muscle groups: quadriceps, glutes, hamstrings ). Step forward with one leg and lower your body until both knees are bent at 90-degree angles. Push back up to the starting position.'),
 (0.6854795217514038,
  'squat ( muscle groups: quadriceps, glutes, lower back ). Stand with feet shoulder-width apart. Bend knees and hips to lower down as if sitting back into a chair, then return to standing.'),
 (0.6720908880233765,
  'hip thrust ( muscle groups: glutes, hamstrings ). Sit on the ground with a bench behind you. Place a barbell over your hips and thrust upward, pushing your hips towards the sky.'),
 (0.6710038185119629,
  'front squat ( muscle groups: quadriceps, glutes, upper back ). Stand with feet shoulder-width apart, holding a barbell in fr

In [24]:
exercises_text = "\n\n".join([e for _, e in scores[:10]])
print(exercises_text)

leg press ( muscle groups: quadriceps, glutes, hamstrings ). Sit in a leg press machine with feet hip-width apart on the platform. Extend legs fully, then return to starting position.

lunges ( muscle groups: quadriceps, glutes, hamstrings ). Step forward with one leg and lower your body until both knees are bent at 90-degree angles. Push back up to the starting position.

squat ( muscle groups: quadriceps, glutes, lower back ). Stand with feet shoulder-width apart. Bend knees and hips to lower down as if sitting back into a chair, then return to standing.

hip thrust ( muscle groups: glutes, hamstrings ). Sit on the ground with a bench behind you. Place a barbell over your hips and thrust upward, pushing your hips towards the sky.

front squat ( muscle groups: quadriceps, glutes, upper back ). Stand with feet shoulder-width apart, holding a barbell in front of your shoulders. Lower into a squat, then return to standing.

hack squat ( muscle groups: quadriceps, glutes, calves ). Stand 

In [25]:
prompt_text = prompt_template.format(exercises_text=exercises_text)
print(prompt_text)


Write a workout plan for training legs - glutes, quadriceps and hamstrings
Use the format:

exercise - sets x reps

Choose from the exercises:
```
leg press ( muscle groups: quadriceps, glutes, hamstrings ). Sit in a leg press machine with feet hip-width apart on the platform. Extend legs fully, then return to starting position.

lunges ( muscle groups: quadriceps, glutes, hamstrings ). Step forward with one leg and lower your body until both knees are bent at 90-degree angles. Push back up to the starting position.

squat ( muscle groups: quadriceps, glutes, lower back ). Stand with feet shoulder-width apart. Bend knees and hips to lower down as if sitting back into a chair, then return to standing.

hip thrust ( muscle groups: glutes, hamstrings ). Sit on the ground with a bench behind you. Place a barbell over your hips and thrust upward, pushing your hips towards the sky.

front squat ( muscle groups: quadriceps, glutes, upper back ). Stand with feet shoulder-width apart, holding 

In [26]:
SYSTEM_PROMPT = """
You're an expert BodyBuilding Coach with 20+ years of experience with
training all types of atheletes. You always follow foundational principles when
designing workout plans. Each workout must have 4 exercises at maximum.
"""

prompt_text = SYSTEM_PROMPT + prompt_text

messages = [
    {"role": "user", "content": prompt_text},
]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
print(prompt)

<s>[INST] 
You're an expert BodyBuilding Coach with 20+ years of experience with
training all types of atheletes. You always follow foundational principles when
designing workout plans. Each workout must have 4 exercises at maximum.

Write a workout plan for training legs - glutes, quadriceps and hamstrings
Use the format:

exercise - sets x reps

Choose from the exercises:
```
leg press ( muscle groups: quadriceps, glutes, hamstrings ). Sit in a leg press machine with feet hip-width apart on the platform. Extend legs fully, then return to starting position.

lunges ( muscle groups: quadriceps, glutes, hamstrings ). Step forward with one leg and lower your body until both knees are bent at 90-degree angles. Push back up to the starting position.

squat ( muscle groups: quadriceps, glutes, lower back ). Stand with feet shoulder-width apart. Bend knees and hips to lower down as if sitting back into a chair, then return to standing.

hip thrust ( muscle groups: glutes, hamstrings ). Sit o

In [27]:
output = llm(prompt)

Here is a sample workout plan for training legs - glutes, quadriceps, and hamstrings:

1. Leg Press - 3 sets of 8-10 reps
2. Romanian Deadlift - 3 sets of 8-10 reps
3. Seated Leg Extension - 3 sets of 12-15 reps
4. Hip Thrust - 3 sets of 12-15 reps

This workout plan includes four exercises that target different parts of the legs, including the quadriceps, glutes, and hamstrings. The leg press targets all three muscle groups simultaneously, while the Romanian deadlift focuses specifically on the hamstrings and lower back. The seated leg extension targets the quadriceps, and the hip thrust targets the glutes and hamstrings. This workout plan can be adjusted based on individual fitness levels and goals.
